In [2]:
#!pip install numpy==1.22
#!pip install pandas==1.3.4
!pip install keras==2.12.0
!pip install daal==2021.4.0

  Using cached daal-2021.4.0-py2.py3-none-manylinux1_x86_64.whl (295.5 MB)
  Using cached tbb-2021.9.0-py2.py3-none-manylinux1_x86_64.whl (4.0 MB)
  Attempting uninstall: tbb
    Found existing installation: TBB 0.2
ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import numpy as np

2023-05-01 18:12:35.171493: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 18:12:35.650791: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-01 18:12:35.653209: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-01 18:12:38.447472: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
#download the data and read it into a pandas dataframe.
concrete_data = pd.read_csv('https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/DL0101EN/labs/data/concrete_data.csv')
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [5]:
concrete_data.shape
concrete_data.describe()
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

In [6]:
#Split data into predictors and target

concrete_data_columns = concrete_data.columns

predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

predictors.head()
#predictors.shape
#target.shape

n_cols = predictors.shape[1] # number of predictors
n_cols


8

## Normalize Data



The next step is to normalize the data by substracting the mean and dividing by the standard deviation.


In [37]:
predictors_norm = (predictors - predictors.mean()) / predictors.std()
target_norm = (target - target.mean()) / target.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


Let's define a function that defines our regression model for us so that we can conveniently call it to create our model.


In [25]:
from keras import Sequential
from tensorflow.keras import layers
from keras.layers import Dense

In [26]:
# define regression model
def regression_model():
    # create model
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    #model.add(Dense(5, activation='relu'))
    #model.add(Dense(5, activation='relu'))
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

1)Randomly split the data into a training and test sets by holding 30% of the data for testing. 

In [38]:
from sklearn.model_selection import train_test_split

predictors_train, predictors_test, target_train, target_test = train_test_split(predictors_norm, target_norm, test_size=0.30,random_state=42)

#predictors_train.shape
target_test.shape
np.array(target_test).reshape(103, 3)#use reshape 103*3 = 309 original size
                                      #for the arg 'multiouput' in mean_squared_error function


array([[ 1.0231236 ,  1.20210398,  2.31549361],
       [-0.03100498, -1.51313011,  0.50653475],
       [-0.72597561,  0.57178178,  0.0947003 ],
       [ 0.78009339, -0.64157349, -1.50175677],
       [ 0.26230735,  0.83696006, -0.16808359],
       [-0.8319272 ,  0.25392699, -0.98576652],
       [ 0.08093258, -0.16329482, -0.26565484],
       [ 0.12822171,  0.77231163, -1.73101926],
       [ 0.16653189, -0.29738046, -1.47541853],
       [ 0.31318806,  1.02611658, -1.35270623],
       [ 0.98660921,  0.22639155,  0.96027096],
       [ 1.83182758, -1.24974761, -0.167485  ],
       [-0.25009133,  0.21561681, -1.32636798],
       [ 0.96625692, -1.21143743, -1.76813225],
       [ 0.13480627,  0.86329831, -1.54365854],
       [ 1.65105141,  1.1811531 ,  0.01987573],
       [-0.84210334, -1.85492876,  1.28052013],
       [ 0.71604355, -0.38357836, -1.40358694],
       [ 1.6881644 , -0.08487867, -0.62541138],
       [-1.32337499, -0.24530255, -0.96062547],
       [ 0.21561681, -1.33474833, -0.028

## Train and Test the Network


In [39]:
#Build the model

model = regression_model()


2)  Train the model on the training data using 50 epochs

In [40]:
# train the model with fit
model.fit(predictors_train, target_train, validation_split=0.3, epochs=50,verbose=2)

Epoch 1/50
16/16 - 1s - loss: 2.6645 - val_loss: 1.8973 - 1s/epoch - 72ms/step
Epoch 2/50
16/16 - 0s - loss: 2.2687 - val_loss: 1.6025 - 110ms/epoch - 7ms/step
Epoch 3/50
16/16 - 0s - loss: 1.9445 - val_loss: 1.3739 - 106ms/epoch - 7ms/step
Epoch 4/50
16/16 - 0s - loss: 1.6881 - val_loss: 1.1876 - 106ms/epoch - 7ms/step
Epoch 5/50
16/16 - 0s - loss: 1.4845 - val_loss: 1.0421 - 104ms/epoch - 7ms/step
Epoch 6/50
16/16 - 0s - loss: 1.3150 - val_loss: 0.9311 - 106ms/epoch - 7ms/step
Epoch 7/50
16/16 - 0s - loss: 1.1831 - val_loss: 0.8413 - 104ms/epoch - 6ms/step
Epoch 8/50
16/16 - 0s - loss: 1.0688 - val_loss: 0.7704 - 105ms/epoch - 7ms/step
Epoch 9/50
16/16 - 0s - loss: 0.9796 - val_loss: 0.7098 - 104ms/epoch - 6ms/step
Epoch 10/50
16/16 - 0s - loss: 0.9018 - val_loss: 0.6630 - 105ms/epoch - 7ms/step
Epoch 11/50
16/16 - 0s - loss: 0.8376 - val_loss: 0.6254 - 104ms/epoch - 6ms/step
Epoch 12/50
16/16 - 0s - loss: 0.7848 - val_loss: 0.5940 - 102ms/epoch - 6ms/step
Epoch 13/50
16/16 - 0s - lo

In [41]:
#Predictions

predictions = model.predict(predictors_test)
np.array(predictions).reshape(103, 3) #use reshape 103*3 = 309 original size
                                      #for the arg 'multiouput' in mean_squared_error function


10/10 [==============================] - 0s 3ms/step


array([[ 0.96980554,  1.0202869 ,  1.6038091 ],
       [ 0.99177116, -1.3765032 ,  0.3397471 ],
       [-0.4901288 ,  0.42189687, -0.18592781],
       [ 0.39224845,  0.06600911, -1.137372  ],
       [ 1.5968497 ,  1.0024209 , -0.57686764],
       [-0.08458834, -0.31111762, -0.82625896],
       [ 0.06715279, -0.46400234, -0.5299642 ],
       [-0.27899224,  0.09749425, -0.9112882 ],
       [-0.113709  ,  0.04872375, -1.2167368 ],
       [ 0.22988507,  0.3321975 , -0.6904157 ],
       [ 0.05715407,  0.15925962,  0.4889385 ],
       [ 0.7004849 , -0.6382229 ,  0.01199792],
       [-0.24936652,  0.3432523 , -1.153141  ],
       [ 0.13506454, -0.81544894, -1.3784287 ],
       [-0.47115755,  0.5978773 , -1.0753561 ],
       [ 1.2760673 ,  0.20797208,  0.701721  ],
       [-1.0294151 , -1.2276368 ,  0.35463884],
       [-0.02119773, -0.05808442, -0.80879533],
       [ 0.2753564 ,  1.0151682 , -0.31530312],
       [-1.0693085 , -0.15611354, -0.8154618 ],
       [ 0.32611036, -0.16925043, -0.061

3) Evaluate the model on the test data and compute the mean squared error between the predicted concrete strength and the actual concrete strength. You can use the mean_squared_error function from Scikit-learn.


In [42]:
from sklearn.metrics import mean_squared_error

In [43]:
metrics = mean_squared_error(target_test, predictions, multioutput = 'raw_values')
metrics

array([0.39463951])

4. Repeat steps 1 - 3, 50 times, i.e., create a list of 50 mean squared errors.

In [44]:
metrics1 = np.zeros(50) #Incializando array
for i in range(49):
    model.fit(predictors_train, target_train, validation_split=0.3, epochs=50,verbose=2)
    predictions = model.predict(predictors_test)
    metrics1[i] = np.array(mean_squared_error(target_test, predictions, multioutput = 'raw_values'))
    print(metrics1[i])

Epoch 1/50
16/16 - 0s - loss: 0.3332 - val_loss: 0.3449 - 163ms/epoch - 10ms/step
Epoch 2/50
16/16 - 0s - loss: 0.3297 - val_loss: 0.3429 - 98ms/epoch - 6ms/step
Epoch 3/50
16/16 - 0s - loss: 0.3262 - val_loss: 0.3409 - 97ms/epoch - 6ms/step
Epoch 4/50
16/16 - 0s - loss: 0.3231 - val_loss: 0.3394 - 98ms/epoch - 6ms/step
Epoch 5/50
16/16 - 0s - loss: 0.3199 - val_loss: 0.3368 - 96ms/epoch - 6ms/step
Epoch 6/50
16/16 - 0s - loss: 0.3167 - val_loss: 0.3358 - 101ms/epoch - 6ms/step
Epoch 7/50
16/16 - 0s - loss: 0.3141 - val_loss: 0.3340 - 100ms/epoch - 6ms/step
Epoch 8/50
16/16 - 0s - loss: 0.3106 - val_loss: 0.3314 - 106ms/epoch - 7ms/step
Epoch 9/50
16/16 - 0s - loss: 0.3078 - val_loss: 0.3307 - 106ms/epoch - 7ms/step
Epoch 10/50
16/16 - 0s - loss: 0.3050 - val_loss: 0.3287 - 104ms/epoch - 6ms/step
Epoch 11/50
16/16 - 0s - loss: 0.3030 - val_loss: 0.3282 - 104ms/epoch - 6ms/step
Epoch 12/50
16/16 - 0s - loss: 0.3003 - val_loss: 0.3266 - 103ms/epoch - 6ms/step
Epoch 13/50
16/16 - 0s - los

In [45]:
#array with 
metrics1

array([0.31566747, 0.23753436, 0.19038729, 0.17541467, 0.17063563,
       0.16787619, 0.16727351, 0.16647662, 0.16603271, 0.16449562,
       0.16569542, 0.16699469, 0.16472272, 0.16526444, 0.16290255,
       0.16323682, 0.16432262, 0.16318803, 0.16443186, 0.16391145,
       0.16464234, 0.1639407 , 0.16500221, 0.1653589 , 0.16410256,
       0.16656502, 0.16540783, 0.16597226, 0.16500377, 0.16661393,
       0.16524769, 0.16425851, 0.16815596, 0.16559345, 0.16616144,
       0.16633553, 0.16805902, 0.16682172, 0.16742565, 0.1678759 ,
       0.16728213, 0.16714534, 0.16834802, 0.16687943, 0.1673137 ,
       0.16854389, 0.1697165 , 0.16833674, 0.16858244, 0.        ])

5. Report the mean and the standard deviation of the mean squared errors.

In [46]:
#The mean value

from statistics import mean

metrics1_avg = mean(metrics1)
metrics1_std = np.std(metrics1)

print("The mean of the MSN array is: ", metrics1_avg, " and the standard deviation is: ", metrics1_std)


The mean of the MSN array is:  0.16794314479501243  and the standard deviation is:  0.03337986846183359
